# Spark_Basics.ipynb

## 簡介 (Introduction)

本 Notebook 將示範使用 **PySpark** 進行資料讀取、基本轉換以及簡單分析的操作。透過此範例，您將學習如何：

- 初始化 SparkSession（PySpark 的入口點）
- 載入資料（CSV、JSON）
- 利用 Pandas DataFrame 與 Spark DataFrame 互相轉換
- 對資料進行基本的轉換與清理（選擇欄位、篩選、建立衍生欄位、分組聚合）
- 使用 Spark 的 DataFrame API 進行簡單分析
- 將分析結果儲存回檔案系統
- 展示一些基本的性能優化手法（例如快取）

請確保您已安裝好 PySpark。
若未安裝，可透過：
```bash
!pip install pyspark
```

## 1. 初始化 SparkSession 與環境設定

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, count, max, min, mean, stddev, expr, upper
from pyspark.sql.types import *

# 建立 SparkSession
# 可加入額外設定，例如 offHeap memory 設定
spark = SparkSession.builder \
    .appName("Spark_Basics_Demo") \
    .config("spark.memory.offHeap.enabled", "true") \
    .config("spark.memory.offHeap.size","1g") \
    .getOrCreate()

spark

## 2. 資料讀取

此處將示範如何從 CSV 與 JSON 檔案讀取資料，以及從 Pandas DataFrame 建立 Spark DataFrame。


### 2.1 讀取 CSV 檔案

In [ ]:
# 假設有一個 sample_data.csv 檔案
# 若無此檔案，可執行以下程式碼建立範例檔案
with open("sample_data.csv", "w") as f:
    f.write("id,name,age,city\n")
    f.write("1,Alice,25,New York\n")
    f.write("2,Bob,30,Los Angeles\n")
    f.write("3,Charlie,35,Chicago\n")
    f.write("4,David,40,New York\n")
    f.write("5,Alice,28,Chicago")

In [ ]:
# 讀取 CSV 檔案
df_csv = spark.read.csv("sample_data.csv", header=True, inferSchema=True)
df_csv.show()
df_csv.printSchema()

### 2.2 讀取 JSON 檔案
此示例假設有一個 `example.json`。若無法檔案可視需要自行建立。

In [ ]:
# 範例 JSON 檔案 (若無可自行建立)：
with open("example.json", "w") as f:
    f.write('[{"id":101, "item":"Book", "price":12.5},\n')
    f.write('{"id":102, "item":"Pen", "price":1.5},\n')
    f.write('{"id":103, "item":"Laptop", "price":900.0}]')

# 讀取 JSON
df_json = spark.read.option("multiline","true").json("example.json")
df_json.show()
df_json.printSchema()

### 2.3 從 Pandas DataFrame 轉換為 Spark DataFrame


In [ ]:
import pandas as pd
import numpy as np

# 建立範例 Pandas DataFrame
pandas_df = pd.DataFrame({
    'id': range(10),
    'value': np.random.randn(10),
    'category': np.random.choice(['A','B','C'], 10)
})

# 轉換為 Spark DataFrame
spark_df = spark.createDataFrame(pandas_df)
spark_df.show()

## 3. 資料轉換與篩選
利用 df_csv 進行示範。

In [ ]:
# 選擇特定欄位
df_selected = df_csv.select("name", "age")
df_selected.show()

# 篩選 age > 30
df_filtered = df_csv.filter(col("age") > 30)
df_filtered.show()

# 建立新欄位 age_plus_one = age + 1
df_newcol = df_csv.withColumn("age_plus_one", col("age") + 1)
df_newcol.show()

## 4. 分組聚合與簡單分析


In [ ]:
# 根據 city 分組，計算該 city 的人數
df_count_by_city = df_csv.groupBy("city").count()
df_count_by_city.show()

# 根據 name 分組，計算平均年齡
df_avg_age_by_name = df_csv.groupBy("name").agg(avg("age").alias("avg_age"))
df_avg_age_by_name.show()


可嘗試更複雜的聚合，如計算 stddev 或 median（使用 approximate 方法）：

In [ ]:
# 計算每個城市的年齡統計
df_city_stats = df_csv.groupBy("city").agg(
    count("id").alias("count_id"),
    avg("age").alias("avg_age"),
    stddev("age").alias("std_age"),
    expr("percentile_approx(age, 0.5)").alias("median_age")
)
df_city_stats.show()

## 5. 使用 SQL 查詢

可以將 DataFrame 註冊為臨時視圖，然後使用 SQL 語法對其進行查詢。

In [ ]:
df_csv.createOrReplaceTempView("people")
sql_result = spark.sql("SELECT city, AVG(age) as avg_age FROM people GROUP BY city ORDER BY avg_age DESC")
sql_result.show()

## 6. 資料輸出
將結果輸出為 CSV 或 Parquet 等格式。

In [ ]:
# 輸出為 CSV
df_avg_age_by_name.coalesce(1).write.mode('overwrite').option('header','true').csv('output_csv')

# 輸出為 Parquet
df_city_stats.write.mode('overwrite').parquet('output_parquet')

## 7. 與 Pandas DataFrame 互轉

可將 Spark DataFrame 轉為 Pandas DataFrame，以進行本地分析或視覺化。

In [ ]:
pandas_result = df_avg_age_by_name.toPandas()
pandas_result

## 8. 性能優化示例

### 8.1 快取 DataFrame
在反覆使用同一 DataFrame 時，可以透過 `cache()` 來快取資料以加速後續動作。

In [ ]:
# 將 df_csv 快取
df_csv.cache()
# 執行動作，觸發快取
df_csv.count()

### 8.2 分區調整
可透過 `repartition()` 改變分區數量，以平衡工作負載。

In [ ]:
df_repart = df_csv.repartition(4)
print("分區數：", df_repart.rdd.getNumPartitions())

## 9. 結束 SparkSession

在所有操作完成後，呼叫 `spark.stop()` 以釋放資源。

In [ ]:
spark.stop()

## 總結

本 Notebook 展示了：
- 建立 SparkSession
- 從 CSV、JSON 載入資料，從 Pandas DF 轉為 Spark DF
- 資料的選擇、篩選、衍生欄位新增
- 分組聚合與統計分析
- 使用 Spark SQL 查詢資料
- 將處理結果輸出成檔案
- 簡單的快取與分區操作優化性能

透過這些步驟，你應該已對 PySpark 的基本用法有初步了解，可作為後續更進階大數據處理、機器學習與分析的基石。